In [39]:
import pandas as pd
import csv
import io
import os

from langchain_openai import ChatOpenAI
from deepagents import create_deep_agent
from deepagents.backends.filesystem import FilesystemBackend
from langchain.agents.middleware import SummarizationMiddleware

import uuid

from langgraph.checkpoint.memory import InMemorySaver

from langchain.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, MessageGraph, StateGraph
from langchain_experimental.utilities import PythonREPL


from dotenv import load_dotenv

In [40]:
load_dotenv()

True

In [41]:
llm = ChatOpenAI(
    model="gpt-4.1-mini",
)

In [42]:
from pathlib import Path

ROOT = Path(r"C:\Users\ANDRES\Desktop\RappIntelligence").resolve()

@tool
def safe_read_file(file_path: str, limit: int = 2000):
    """
    Lee archivos SOLO dentro de ROOT.
    Bloquea rutas absolutas tipo /data/... o C:\\...
    """
    p = Path(file_path)

    # Bloquear rutas absolutas o que intenten escapar
    if p.is_absolute() or ".." in p.parts:
        return {"error": f"Ruta no permitida: {file_path}"}

    abs_path = (ROOT / p).resolve()

    # Verificar que quede dentro de ROOT
    if not str(abs_path).startswith(str(ROOT)):
        return {"error": f"Ruta fuera del proyecto: {abs_path}"}

    if not abs_path.exists():
        return {"error": f"Archivo no encontrado: {abs_path}"}

    with open(abs_path, "r", encoding="utf-8") as f:
        content = f.read(limit)

    return {"path": str(abs_path), "content": content}

In [43]:
#backend = LocalShellBackend(root_dir=str(ROOT), env=os.environ.copy())

In [44]:
checkpointer = InMemorySaver()

In [58]:
load_data = {
    "name": "Cargador_de_archivos",
    "description": "subagente que carga los archivos csv",
    "system_prompt": """Tu unica tarea es revisar los archivos csv y cargarlos si la solicitud del usuario lo requiere.
    No es necesario que siempre cargues todos los archivos, debes evaluar la solicitud del usuario y cargar solo los archivos
    que sean necesarios.""",
    
    "tools": [safe_read_file],
    "model": "openai:gpt-4.1-mini",
}

In [59]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()

MAX_OUTPUT_CHARS = 3000
BLACKLIST_TERMS = ["import os", "import sys", "__import__", "open(", "subprocess", "socket", "requests", "eval(", "exec(", "os.system"]

@tool
def python_repl_tool(code: str) -> str:
    """
    Ejecuta `code` en un REPL con un prelude que expone helpers: load_csv, preview, schema_stats.
    Se bloquean patterns peligrosos y se trunca la salida.
    """
    # chequeo blacklist simple
    low = code.lower()
    for b in BLACKLIST_TERMS:
        if b in low:
            return f"ERROR: uso de terminos prohibidos: {b}"

    # PRELUDE: helpers disponibles para el LLM (puede ajustar)
    prelude = f'''
from pathlib import Path
import pandas as pd
ROOT = r"{str(ROOT)}"

def load_csv(name, usecols=None, nrows=None):
    p = Path(ROOT) / name
    if not str(p.resolve()).startswith(str(Path(ROOT))):
        raise Exception("ruta fuera del proyecto")
    return pd.read_csv(p, usecols=usecols, nrows=nrows)

def preview(name, n=5):
    return load_csv(name, nrows=n).to_json(orient="records")

def schema_stats(name, n=1000):
    df = load_csv(name, nrows=n)
    return {{
        "dtypes": df.dtypes.apply(lambda x: str(x)).to_dict(),
        "nnulls": df.isna().sum().to_dict(),
        "numeric_sample": df.select_dtypes("number").agg(["min","max","mean"]).to_dict()
    }}
'''

    full_code = prelude + "\n" + code

    try:
        out = python_repl.run(full_code)
        if len(out) > MAX_OUTPUT_CHARS:
            return out[:MAX_OUTPUT_CHARS] + "\n\n...[truncated]"
        return out
    except Exception as e:
        return f"ERROR: {e}"

In [60]:
analysis_data = {
    "name": "Analizador_de_los_datos",
    "description": "subagente que analiza los datos",
    "system_prompt": """Tu unica tarea es hacer el analisis correspondiente de los datos segun la solicitud del usuario""",
    
    "tools": [python_repl_tool],
    "model": "openai:gpt-4.1-mini",
}

In [61]:
subagents = [load_data, analysis_data]

In [62]:
system_prompt = """
Eres un analista de datos con experiencia en análisis de métricas y órdenes.
Tu tarea es analizar los datos dependiendo las preguntas que haga el usuario.
"""

In [63]:
agent = create_deep_agent(
    model=llm,
    #tools=[safe_read_file],
    backend=FilesystemBackend(root_dir="C:/Users/ANDRES/Desktop/RappIntelligence"),
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(
            model="gpt-4.1-mini",
            trigger=("tokens", 10000),
            keep=("messages", 20),
        ),
    ],
    interrupt_on={
        "write_file": True,  # Default: approve, edit, reject
        "read_file": True,  # No interrupts needed
        "edit_file": True    # Default: approve, edit, reject
    },
    subagents=subagents,
    system_prompt=system_prompt,
)

thread_id = str(uuid.uuid4())
config={"configurable": {"thread_id": thread_id}}

C:\Users\ANDRES\AppData\Local\Temp\ipykernel_7936\1459261890.py:4: DeprecationWarning: FilesystemBackend virtual_mode default will change in deepagents 0.5.0; please specify virtual_mode explicitly. Note: virtual_mode is for virtual path semantics (e.g., CompositeBackend routing) and optional path-based guardrails; it does not provide sandboxing or process isolation. Security note: leaving virtual_mode=False allows absolute paths and '..' to bypass root_dir. Consult the API reference for details.
  backend=FilesystemBackend(root_dir="C:/Users/ANDRES/Desktop/RappIntelligence"),


In [64]:
input_message = {
    "role": "user",
    "content": ("Cuáles son las 5 zonas con mayor % Lead Penetration esta semana"
    )
}

In [65]:
for step in agent.stream(
    {"messages": [input_message]},
    config,
    stream_mode="updates",
):
    for _, update in step.items():
        if update and (
            (isinstance(update, dict) and (messages := update.get("messages"))) or
            (isinstance(update, tuple) and update[0] == "messages" and (messages := update[1]))
        ) and isinstance(messages, list):
            for message in messages:
                message.pretty_print()
                print("TOKENS:", getattr(message, "usage_metadata", None) or message.additional_kwargs.get("usage", None))

================================== Ai Message ==================================
Tool Calls:
  task (call_m866fJw4UNxEFrhQ24VZcKuR)
 Call ID: call_m866fJw4UNxEFrhQ24VZcKuR
  Args:
    description: Analizar los datos de esta semana para identificar las 5 zonas con mayor porcentaje de Lead Penetration. Se requiere un informe con las zonas ordenadas de mayor a menor porcentaje.
    subagent_type: Analizador_de_los_datos
TOKENS: {'input_tokens': 5095, 'output_tokens': 60, 'total_tokens': 5155, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
================================= Tool Message =================================
Name: task

No he encontrado un archivo específico para los datos de esta semana en los directorios comunes. Por favor, proporcióname el archivo o datos donde pueda realizar el análisis del porcentaje de Lead Penetration por zona para identificar las 5 zonas con mayor porcentaje.
TOKENS: None
======================